Criando um sistema de recomendação de filmes usando Machine Learning, processo semelhante as plataformas da Netflix, Apple Music, Spotify, entre outros.

fonte: https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

crédito: https://www.youtube.com/watch?v=sDI3ntGFbSI&t=2029s&ab_channel=NerddosDados

In [62]:
import pandas as pd
import numpy as np

Importando os arquivos com os filmes e avaliações.

In [63]:
filmes = pd.read_csv('movies_metadata.csv', low_memory=False)
filmes.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [64]:
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


Fazendo o pré processamento dos dados do DataFrame "filmes". Vamos filtrar somente as colunas necessárias e renomear o nome das vairáveis.

In [65]:
filmes = filmes [['id', 'original_title', 'original_language', 'vote_count']]
filmes.rename(columns = {'id': 'ID_FILME', 'original_title': 'TITULO',
                         'original_language': 'LINGUAGEM',
                         'vote_count': 'QT_AVALIACOES'}, inplace=True)
filmes.head()

<ipython-input-65-8eea0f4b05b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes.rename(columns = {'id': 'ID_FILME', 'original_title': 'TITULO',


,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


Fazendo o pré processamento dos dados do DataFrame "avaliacoes". Vamos filtrar somente as colunas necessárias e renomear o nome das vairáveis.

In [66]:
avaliacoes = avaliacoes [['userId', 'movieId', 'rating']]
avaliacoes.rename(columns = {'userId': 'ID_USUARIO', 'movieId': 'ID_FILME',
                             'rating': 'AVALIACAO'}, inplace=True)
avaliacoes.head()

<ipython-input-66-f7d23bb6356f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avaliacoes.rename(columns = {'userId': 'ID_USUARIO', 'movieId': 'ID_FILME',


,ID_USUARIO,ID_FILME,AVALIACAO
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


verificando e removendo os valores nulos

In [67]:
filmes.isna().sum()

ID_FILME          0
TITULO            0
LINGUAGEM        11
QT_AVALIACOES     6
dtype: int64

In [68]:
filmes.dropna(inplace=True)

In [69]:
filmes.isna().sum()

ID_FILME         0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [70]:
avaliacoes.isna().sum()

ID_USUARIO    0
ID_FILME      0
AVALIACAO     0
dtype: int64

Verificando a quantidade de avaliações por usuário. Vamos considerar como base as pessoas que fizeram muitas avaliações para ter um peso ideal no modelo preditivo.

In [71]:
avaliacoes['ID_USUARIO'].value_counts()

45811     18276
8659       9279
270123     7638
179792     7515
228291     7410
          ...  
30155         1
9641          1
164717        1
243426        1
234625        1
Name: ID_USUARIO, Length: 270896, dtype: int64

Iremos selecionar apenas os usuários que fizeram mais de 999 avaliações.

In [72]:
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

(2509,)

In [73]:
y

Int64Index([ 45811,   8659, 270123, 179792, 228291, 243443,  98415, 229879,
             98787, 172224,
            ...
            227649, 244253, 257117,  30733, 196384,  53075, 220764, 214328,
             14354, 182812],
           dtype='int64', length=2509)

Visualizando o tamanho do dataset "avaliacoes".

In [74]:
avaliacoes.shape

(26024289, 3)

Iremos pegar somente avaliações dos usuários que contém ID igual ao "y"

In [75]:
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]

In [76]:
avaliacoes.shape

(3844582, 3)

In [77]:
avaliacoes.head(5)

,ID_USUARIO,ID_FILME,AVALIACAO
17291,229,1,3.0
17292,229,2,3.0
17293,229,4,2.0
17294,229,5,1.0
17295,229,7,2.0


In [78]:
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


Vamos pegar os filmes que receberam uma grande quantidade de avaliação

In [79]:
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

Vamos agrupar e visualizar a quantidade de filmes pela linguagem

In [80]:
filmes_linguagem = filmes['LINGUAGEM'].value_counts()
filmes_linguagem.head(20)

en    1100
fr       5
ja       5
it       3
ko       2
pt       1
de       1
es       1
cn       1
sv       1
id       1
Name: LINGUAGEM, dtype: int64

Selecionando somente os filmes em inglês

In [81]:
filmes = filmes[filmes['LINGUAGEM'] == 'en']

Visualizando os tipos de dados das variáveis

In [82]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   object 
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB


In [83]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3844582 entries, 17291 to 26023521
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID_USUARIO  int64  
 1   ID_FILME    int64  
 2   AVALIACAO   float64
dtypes: float64(1), int64(2)
memory usage: 117.3 MB


O ID de filmes está como objeto, iremos converter para inteiro

In [84]:
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

In [85]:
filmes.shape

(1100, 4)

In [86]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   int64  
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 43.0+ KB


Concatenando os dataframes e verificando a quantidade de filmes com avaliações pelo tamanho do arquivo

In [87]:
avaliacoes_e_filmes = avaliacoes.merge(filmes, on = 'ID_FILME')
avaliacoes_e_filmes.head()

,ID_USUARIO,ID_FILME,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
0,229,12,1.0,Finding Nemo,en,6292.0
1,741,12,3.0,Finding Nemo,en,6292.0
2,1932,12,0.5,Finding Nemo,en,6292.0
3,3437,12,2.0,Finding Nemo,en,6292.0
4,3694,12,1.5,Finding Nemo,en,6292.0


In [88]:
avaliacoes_e_filmes.shape

(189882, 6)

In [89]:
avaliacoes_e_filmes.isna().sum()

ID_USUARIO       0
ID_FILME         0
AVALIACAO        0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

Vamos descartar os valores duplicados, para que não tenha problemas de termos o mesmo usuário avaliando o mesmo filme diversar vezes.

In [90]:
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO', 'ID_FILME'], inplace=True)

In [91]:
avaliacoes_e_filmes.shape

(189882, 6)

Vamos deletar a variável ID_FILME porque não iremos utilizá-la

In [92]:
del avaliacoes_e_filmes['ID_FILME']

In [93]:
avaliacoes_e_filmes.head(20)

,ID_USUARIO,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
0,229,1.0,Finding Nemo,en,6292.0
1,741,3.0,Finding Nemo,en,6292.0
2,1932,0.5,Finding Nemo,en,6292.0
3,3437,2.0,Finding Nemo,en,6292.0
4,3694,1.5,Finding Nemo,en,6292.0
5,3950,1.0,Finding Nemo,en,6292.0
6,4294,2.0,Finding Nemo,en,6292.0
7,4540,2.0,Finding Nemo,en,6292.0
8,4916,3.0,Finding Nemo,en,6292.0
9,4932,2.0,Finding Nemo,en,6292.0


Agora precisamos fazer um PIVOT. O que queremos é que cada ID_USUARIO seja uma variável com o respectivo valor de nota para cada filme avaliado.

In [94]:
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns = 'ID_USUARIO',
                                               index = 'TITULO',
                                               values = 'AVALIACAO')

filmes_pivot.head(20)

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.5,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
127 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN
2 Fast 2 Furious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey,NaN,NaN,3.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN
27 Dresses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28 Days Later,3.0,NaN,2.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,2.5,3.0,NaN,NaN,3.0,NaN
28 Weeks Later,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,...,NaN,0.5,3.0,1.0,2.0,3.0,NaN,2.5,NaN,NaN


Vamos trocar os valores nulos por 0.

In [95]:
filmes_pivot.fillna(0, inplace=True)
filmes_pivot.head()

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.5,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
127 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.5,2.0,0.0,0.0,0.0,0.0
2 Fast 2 Furious,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Importando o csr_matrix do pacote SciPy para possibilitar criarmos uma matriz sparsa

In [96]:
from scipy.sparse import csr_matrix

filmes_sparse = csr_matrix(filmes_pivot)

In [97]:
type(filmes_sparse)

scipy.sparse._csr.csr_matrix

Importando o algoritmo KNN do SciKit Learn, criando e treinando o modelo preditivo.

In [98]:
from sklearn.neighbors import NearestNeighbors

In [99]:
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)

NearestNeighbors(algorithm='brute')

Fazendo previsões de sugestões de filmes

In [100]:
# 127 Hours
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['127 Hours'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
  print(filmes_pivot.index[sugestions[i]])

Index(['127 Hours', 'American Hustle', 'The Expendables 2', 'Lord of War',
       'RED 2'],
      dtype='object', name='TITULO')


In [101]:
# Toy Story
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['Toy Story'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
  print(filmes_pivot.index[sugestions[i]])

Index(['Toy Story', 'Meet the Fockers', 'Top Gun',
       'Harry Potter and the Chamber of Secrets',
       'Austin Powers: International Man of Mystery'],
      dtype='object', name='TITULO')


In [102]:
# 1408
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['1408'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
  print(filmes_pivot.index[sugestions[i]])

Index(['1408', 'Pirates of the Caribbean: At World's End', 'Platoon', 'Snitch',
       'The Expendables 2'],
      dtype='object', name='TITULO')


In [103]:
# 2 Fast 2 Furious
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['2 Fast 2 Furious'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
  print(filmes_pivot.index[sugestions[i]])

Index(['2 Fast 2 Furious', 'Bambi', 'The Matrix Reloaded',
       'Brokeback Mountain', 'Lord of War'],
      dtype='object', name='TITULO')
